### Database connection

In [1]:
from langchain_community.utilities import SQLDatabase

In [2]:
db_path = '../data/inventory.db'
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
print("dialect:", db.dialect)
print("tables:", db.get_usable_table_names())

dialect: sqlite
tables: ['Orders', 'Stock']


In [3]:
records = db.run("SELECT * FROM Stock LIMIT 1")
print(records)

[('1009AA', 7210.0, 'Kg', 30, 48, 28.76326)]


In [4]:
info = db.get_table_info(["Stock"])
print(info)


CREATE TABLE "Stock" (
	id TEXT, 
	current_stock_quantity FLOAT, 
	units TEXT, 
	avg_lead_time_days BIGINT, 
	maximum_lead_time_days BIGINT, 
	unit_price FLOAT
)

/*
3 rows from Stock table:
id	current_stock_quantity	units	avg_lead_time_days	maximum_lead_time_days	unit_price
1009AA	7210.0	Kg	30	48	28.76326
1077CA	46516.0	Kg	45	70	22.9777
1083AA	48210.0	Kg	45	68	29.02
*/


In [5]:
records = db.run("SELECT * FROM Orders LIMIT 1")
print(records)

[('2022-11-19 00:00:00.000000', '3084CA', 1.0)]


In [6]:
info = db.get_table_info(["Orders"])
print(info)



CREATE TABLE "Orders" (
	date DATETIME, 
	id TEXT, 
	quantity FLOAT
)

/*
3 rows from Orders table:
date	id	quantity
2022-11-19 00:00:00	3084CA	1.0
2022-06-30 00:00:00	2621AA	20.0
2023-04-19 00:00:00	1193BA	147.0
*/


### LLM

In [7]:
from langchain_ollama import ChatOllama

#https://ollama.com/library/qwen2.5-coder:0.5b

llm = ChatOllama(model="qwen2.5-coder:0.5b")

### Tools

In [8]:
# tools
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
toolkit

SQLDatabaseToolkit(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000024F42270980>, llm=ChatOllama(model='qwen2.5-coder:0.5b'))

### SQL agent

In [9]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db,agent_type="openai-tools",verbose=True)


#### Example 1

In [10]:
#expected_query = "SELECT COUNT(*) FROM Orders"
#expected result = 33919
query = "How many orders are there in the Orders table?"
response = agent_executor.invoke(query)
print("response:", response)



> Entering new SQL Agent Executor chain...
  Finally, I should use sql_db_query_checker to double check if my query is correct before executing it.

> Finished chain.
response: {'input': 'How many orders are there in the Orders table?', 'output': '  Finally, I should use sql_db_query_checker to double check if my query is correct before executing it.'}


Resultado: KO
La latencia 9.8s

#### Example 2

In [12]:
expected_query= "SELECT * FROM Orders WHERE quantity = 475"
# Example 2
query2 = "Select the orders that have a quantity equal 475 in the Orders table."
response2 = agent_executor.invoke(query2)
print("response:", response2)



> Entering new SQL Agent Executor chain...
  Lastly, I should check my query with sql_db_query_checker before executing it.

> Finished chain.
response: {'input': 'Select the orders that have a quantity equal 475 in the Orders table.', 'output': '  Lastly, I should check my query with sql_db_query_checker before executing it.'}


Resultado: KO
Latencia: 1.6

#### Example 3

In [14]:
expected_query= "SELECT id, date, quantity FROM Orders WHERE quantity = 475 LIMIT 10;"
expected_result=[('2785BA', '2023-04-16 00:00:00.000000', 475.0), 
                 ('1993AA', '2022-05-23 00:00:00.000000', 475.0), 
                 ('2727CA', '2022-05-23 00:00:00.000000', 475.0), 
                 ('2814BA', '2022-04-03 00:00:00.000000', 475.0), 
                 ('2847BA', '2022-09-23 00:00:00.000000', 475.0), 
                 ('2291AA', '2022-08-20 00:00:00.000000', 475.0), 
                 ('2803BA', '2022-03-28 00:00:00.000000', 475.0)]

# Example 2
query3 = "What are the first 10 orders that have a quantity equal 475 in the Orders table."
response3 = agent_executor.invoke(query3)
print("response:", response3)



> Entering new SQL Agent Executor chain...
  Finally, I should query the Orders table to get the first 10 orders that have a quantity equal to 475.  If no orders meet this condition, I will return "I don't know".

> Finished chain.
response: {'input': 'What are the first 10 orders that have a quantity equal 475 in the Orders table.', 'output': '  Finally, I should query the Orders table to get the first 10 orders that have a quantity equal to 475.  If no orders meet this condition, I will return "I don\'t know".'}


Resultado: KO La latencia 2.9s


### Example 4 (CoT + schema)

In [15]:
expected_query= "SELECT id, date, quantity FROM Orders WHERE quantity = 475 LIMIT 10;"
expected_result=[('2785BA', '2023-04-16 00:00:00.000000', 475.0), 
                 ('1993AA', '2022-05-23 00:00:00.000000', 475.0), 
                 ('2727CA', '2022-05-23 00:00:00.000000', 475.0), 
                 ('2814BA', '2022-04-03 00:00:00.000000', 475.0), 
                 ('2847BA', '2022-09-23 00:00:00.000000', 475.0), 
                 ('2291AA', '2022-08-20 00:00:00.000000', 475.0), 
                 ('2803BA', '2022-03-28 00:00:00.000000', 475.0)]

# Example 2
query4 = "What are records that have a quantity equal 475 in the Orders table. The table Orders has the columns: id, date and quantity. Let's think step by step."
response4 = agent_executor.invoke(query4)
print("response:", response4)



> Entering new SQL Agent Executor chain...
  Finally, I should run the query to get the results.

Here's a possible SQL query:

```sql
SELECT id, date, quantity FROM Orders WHERE quantity = 475;
```

This query will return all records from the Orders table where the quantity is equal to 475.

> Finished chain.
response: {'input': "What are records that have a quantity equal 475 in the Orders table. The table Orders has the columns: id, date and quantity. Let's think step by step.", 'output': "  Finally, I should run the query to get the results.\n\nHere's a possible SQL query:\n\n```sql\nSELECT id, date, quantity FROM Orders WHERE quantity = 475;\n```\n\nThis query will return all records from the Orders table where the quantity is equal to 475."}


In [19]:
db.run("SELECT id, date, quantity FROM Orders WHERE quantity = 475;")

"[('2785BA', '2023-04-16 00:00:00.000000', 475.0), ('1993AA', '2022-05-23 00:00:00.000000', 475.0), ('2727CA', '2022-05-23 00:00:00.000000', 475.0), ('2814BA', '2022-04-03 00:00:00.000000', 475.0), ('2847BA', '2022-09-23 00:00:00.000000', 475.0), ('2291AA', '2022-08-20 00:00:00.000000', 475.0), ('2803BA', '2022-03-28 00:00:00.000000', 475.0)]"

Resultado: Correcto. Devuelve la query aunque no la ejecuta. "SELECT id, date, quantity FROM Orders WHERE quantity = 475;"

Parece que ignora el resto de los comentarios sobre el schema.

Latencia 3.8s



### Example 5 (Schema)

In [16]:
expected_query= "SELECT id, date, quantity FROM Orders WHERE quantity = 475 LIMIT 10;"
expected_result=[('2785BA', '2023-04-16 00:00:00.000000', 475.0), 
                 ('1993AA', '2022-05-23 00:00:00.000000', 475.0), 
                 ('2727CA', '2022-05-23 00:00:00.000000', 475.0), 
                 ('2814BA', '2022-04-03 00:00:00.000000', 475.0), 
                 ('2847BA', '2022-09-23 00:00:00.000000', 475.0), 
                 ('2291AA', '2022-08-20 00:00:00.000000', 475.0), 
                 ('2803BA', '2022-03-28 00:00:00.000000', 475.0)]


query5 = "What are records that have a quantity equal 475 in the Orders table. The table Orders has the columns: id, date and quantity."
response5 = agent_executor.invoke(query5)
print("response:", response5)



> Entering new SQL Agent Executor chain...
 Finally, I should run an SQL query that returns the records where the quantity is equal to 475.

> Finished chain.
response: {'input': 'What are records that have a quantity equal 475 in the Orders table. The table Orders has the columns: id, date and quantity.', 'output': ' Finally, I should run an SQL query that returns the records where the quantity is equal to 475.'}


Resultado: KO. 
Latencia: 1.4s

### Example 6 (CoT)

In [17]:
expected_query= "SELECT id, date, quantity FROM Orders WHERE quantity = 475 LIMIT 10;"
expected_result=[('2785BA', '2023-04-16 00:00:00.000000', 475.0), 
                 ('1993AA', '2022-05-23 00:00:00.000000', 475.0), 
                 ('2727CA', '2022-05-23 00:00:00.000000', 475.0), 
                 ('2814BA', '2022-04-03 00:00:00.000000', 475.0), 
                 ('2847BA', '2022-09-23 00:00:00.000000', 475.0), 
                 ('2291AA', '2022-08-20 00:00:00.000000', 475.0), 
                 ('2803BA', '2022-03-28 00:00:00.000000', 475.0)]


query6 = "What are records that have a quantity equal 475 in the Orders table. Let's think step by step."
response6 = agent_executor.invoke(query6)
print("response:", response6)



> Entering new SQL Agent Executor chain...
  Finally, I should query the Orders table for orders with a quantity equal to 475.

Here is the SQL query that accomplishes this:

```sql
SELECT *
FROM Orders
WHERE Quantity = 475;
```

This query selects all columns from the Orders table where the Quantity is equal to 475.

> Finished chain.
response: {'input': "What are records that have a quantity equal 475 in the Orders table. Let's think step by step.", 'output': '  Finally, I should query the Orders table for orders with a quantity equal to 475.\n\nHere is the SQL query that accomplishes this:\n\n```sql\nSELECT *\nFROM Orders\nWHERE Quantity = 475;\n```\n\nThis query selects all columns from the Orders table where the Quantity is equal to 475.'}


Resultado: Devuelve la query correcta pero no es capaz de ejecutarlo.

Latencia 2.7s

In [20]:
db.run("SELECT * FROM Orders WHERE Quantity = 475;")

"[('2023-04-16 00:00:00.000000', '2785BA', 475.0), ('2022-05-23 00:00:00.000000', '1993AA', 475.0), ('2022-05-23 00:00:00.000000', '2727CA', 475.0), ('2022-04-03 00:00:00.000000', '2814BA', 475.0), ('2022-09-23 00:00:00.000000', '2847BA', 475.0), ('2022-08-20 00:00:00.000000', '2291AA', 475.0), ('2022-03-28 00:00:00.000000', '2803BA', 475.0)]"